In [1]:
pip install google-cloud-vision

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install opencv-python

In [3]:
pip install google-cloud-automl

In [4]:
# Librerias
import pandas as pd
import numpy as np
import os
from google.cloud import vision
import io
import cv2
import urllib
from google.cloud import automl
import math

In [5]:
# Aplicar vision api solo a una imagen
def vision_api(url, jason_path, t_product = 0.5, t_face = 0.5):
    # Credenciales
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = jason_path
    # Instanciar cliente
    client = vision.ImageAnnotatorClient()
    
    # Vector de salida (fila) prob. producto, producto, prob. sin rostro cliente, sin rostro cliente, contexto
    tamaño = (1,5)
    result = np.zeros(tamaño)
    
    # Se lee imagen
    image = vision.Image()
    image.source.image_uri = url
    
    # Obtener área total de la imagen
    url_open = urllib.request.urlopen(url)
    image_cv = np.asarray(bytearray(url_open.read()), dtype="uint8")
    image_cv = cv2.imdecode(image_cv, cv2.IMREAD_COLOR)
    area_total = image_cv.shape[0]*image_cv.shape[1]
    
    # Detección de objetos
    objects_response = client.object_localization(image=image)
    print('error1:')
    print(objects_response.error)
    # Contador del while por si hay error de lectura
    i=0
    while objects_response.error.message == 'We can not access the URL currently. Please download the content and pass it in.' \
    and i <5:
        print('Fallo en lectura de URL. Intentando nuevamente.\n')
        objects_response = client.object_localization(image=image)
        i += 1
    
    print('error2:')
    print(objects_response.error)
    objects = objects_response.localized_object_annotations
    print(objects)
    # Ver si en objetos hay algun producto
    # Contador de productos
    contador = 0
    # Inicializar area de producto
    area_producto = 0
    # Inicializar lista de confianzas
    confidence_list = []
    for object_ in objects:
        if (object_.name == 'Luggage & bags') or (object_.name == 'Bagged packaged goods') \
            or (object_.name =='Packaged goods') or (object_.name =='Boxed packaged goods') \
            or (object_.name =='Bag') or (object_.name =='Shipping box'):
            # Guardar confianza de la detección
            confidence_list.append(object_.score)
            # Guardar coordenadas
            coordenadas = []
            for vertex in object_.bounding_poly.normalized_vertices:
                coordenadas.append((vertex.x, vertex.y))          
            # Punto superior izquierdo del rectángulo
            x0 = (int(coordenadas[0][0]*image_cv.shape[1]), int(coordenadas[0][1]*image_cv.shape[0]))
            # Punto inferior derecho del rectángulo
            x1 = (int(coordenadas[2][0]*image_cv.shape[1]), int(coordenadas[2][1]*image_cv.shape[0])) 
            # Sumar áreas de cada producto
            area_producto += abs((x0[0]-x1[0])*(x0[1]-x1[1]))
            contador+=1
     
    if contador != 0:
        # Rellenar con la confianza máxima la posición correspondiente en la fila
        result[0][0] += max(confidence_list)
        
        # Comparación con umbral
        if max(confidence_list) >= t_product:
            # Rellenar con 1 la posición correspondiente en la fila
            result[0][1] += 1       
         
        # Calcular contexto
        # Dividir por área total
        razon_areas = area_producto/area_total
        # Rellenar contexto en fila
        result[0][4] = razon_areas
    
    # Detección de caras
    response = client.face_detection(image=image)
    print('error1:')
    print(response.error)
    # Contador del while por si hay error de lectura
    while response.error.message == 'We can not access the URL currently. Please download the content and pass it in.'\
    and i < 5:
        print('Fallo en lectura de URL. Intentando nuevamente.\n')
        response = client.face_detection(image=image)
        i += 1
    print('error2:')
    print(response.error)
    faces = response.face_annotations
    #from pprint import pprint
    print(response.face_annotations)
    #pprint(dir(client.face_detection(image=image)))
    
    # Inicializar lista de confianzas
    confidence_list = []
    for face in faces:
        # Guardar confianza de la detección
        confidence_list.append(face.detection_confidence)
    
    if len(faces) != 0: # Si ocurrre esto es por que hay detección
        # Rellenar con 1 menos la confianza máxima (confianza de que no haya un rostro) 
        # la posición correspondiente en la fila
        result[0][2] += (1-max(confidence_list))
        
        # Comparación con umbral
        if (1-max(confidence_list)) > 1 - t_face:
            # Rellenar con 1 la posición correspondiente en la matriz
            result[0][3] += 1
    
    # Entonces no hay rostro de cliente 
    else:
        result[0][2] += 1
        result[0][3] += 1    
    
    # Se hace un dataframe
    result_data = pd.DataFrame(result, columns=['Prob. Producto', 'Producto', 'Prob. Sin rostro cliente', 'Sin rostro cliente',\
                                                'Contexto'])
    return result_data    

In [6]:
vision_api('https://firebasestorage.googleapis.com/v0/b/txd-scex-tmgt-geosort-prod.appspot.com/o/delivery%2Fnmunoz%40atenas.cl%2F149048661923.png?alt=media&token=265bb962-37dd-4107-a521-268a2211cd94', "clave2\\tc-sc-bi-bigdata-corp-tsod-dev-739359c1cf2a.json")

error1:
code: 14
message: "We can not access the URL currently. Please download the content and pass it in."

Fallo en lectura de URL. Intentando nuevamente.

error2:

[mid: "/j/5qg9b8"
name: "Packaged goods"
score: 0.847762942314148
bounding_poly {
  normalized_vertices {
    x: 0.01057373359799385
    y: 0.005080696661025286
  }
  normalized_vertices {
    x: 0.9966344833374023
    y: 0.005080696661025286
  }
  normalized_vertices {
    x: 0.9966344833374023
    y: 0.583638072013855
  }
  normalized_vertices {
    x: 0.01057373359799385
    y: 0.583638072013855
  }
}
]
error1:

error2:

[]


,Prob. Producto,Producto,Prob. Sin rostro cliente,Sin rostro cliente,Contexto
0,0.847763,1.0,1.0,1.0,0.568047


In [18]:
# Aplicar AutoMl solo a una imagen
def auto_ml(url, jason_path, threshold = 0.5):
    # Credenciales
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = jason_path
    
    # id del proyeto y del modelo para obtener la dirección donde esta alojado el modelo
    project_id = "tc-sc-bi-bigdata-corp-tsod-dev"
    # si se cambia el modelo, hay que tener cuidado con las etiquetas
    model_id = "ICN5209188124751560704"
    
    # Instanciar cliente
    prediction_client = automl.PredictionServiceClient()
    
    # Obtención dirección donde esta alojado el modelo
    model_full_id = automl.AutoMlClient.model_path(project_id, "us-central1", model_id)
    
    # Leer imagen del url
    url_open = urllib.request.urlopen(url)
    content = url_open.read()
    
    image = automl.Image(image_bytes=content)
    payload = automl.ExamplePayload(image=image)
    
    # Se escoge el valor del treshold
    params = {"score_threshold": str(threshold)}
    
    # Se aplica el modelo
    request = automl.PredictRequest(name=model_full_id, payload=payload, params=params)
    response = prediction_client.predict(request=request)
    
    # Vector de salida (fila) Prob. Etiqueta producto, Etiqueta producto, Prob. Numero domicilio, Numero domicilio
    tamaño = (1,4)
    result = np.zeros(tamaño)
    
    # Se verifica si el modelo identifica la etiqueta del producto y el numero de domicilio,
    # dependiendo de lo anterior se rellena con el 'score' y un 1 o 0 si hay o no detección 
    # respectivamente, segun el umbral escogido.
    for resultado in response.payload:
        if resultado.display_name == 'etiqueta_produto':
            result[0,0] = resultado.classification.score
            result[0,1] += 1
        if resultado.display_name == 'numero_domicilio':
            result[0,2] = resultado.classification.score
            result[0,3] += 1
    
    # Se hace un dataframe
    result_data = pd.DataFrame(result, columns=['Prob. Etiqueta producto', 'Etiqueta producto', 'Prob. Numero domicilio',\
                                                'Numero domicilio'])
    return result_data    

In [20]:
url = 'https://prdadessacorptrl.blob.core.windows.net/cl-images/20220519233512-Falabella-149053383533-1-EVIDENCE.png'
jason="key\\tc-sc-bi-bigdata-corp-tsod-dev-82c15beedd2a.json"
auto_ml(url, jason)

payload {
  annotation_spec_id: "8731840311459840000"
  classification {
    score: 1.0
  }
  display_name: "etiqueta_produto"
}



,Prob. Etiqueta producto,Etiqueta producto,Prob. Numero domicilio,Numero domicilio
0,1.0,1.0,0.0,0.0


In [9]:
# Función para calcular score
def calculo_score(data_frame):
    array = np.array(data_frame)
    # Inicialización de los pesos
    w_product = 0.3
    w_without_face = 0.1
    w_context = 0.2
    w_product_label = 0.3
    w_address_number = 0.1
    # Función para mapear la importancia del contexto
    k = 55.3
    new_context = math.exp(-(data_frame['Contexto'][0]-0.5)**2*k)
    # Se hace un dataframe con los valores obtenidos
    score = data_frame['Producto'][0]*w_product + data_frame['Sin rostro cliente'][0]*w_without_face + \
    new_context*w_context + data_frame['Etiqueta producto'][0]*w_product_label + \
    data_frame['Numero domicilio'][0]*w_address_number
    
    return score

In [21]:
# Función que retorna las confidencias + el score en un dataframe
def full_model(url,jason_path_api, jason_path_automl):
    # Se aplica vision_api y auto_ml
    df_1 = vision_api(url, jason_path_api)
    df_2 = auto_ml(url, jason_path_automl)
    # Se contanenan resultados
    df_concatenado = pd.concat([df_1, df_2], axis=1)
    # Se calcula el score
    score = calculo_score(df_concatenado)
    # Se concatena todo lo anterior en un dataframe
    df_score = pd.DataFrame(np.array([score]), columns=['Score'])
    return pd.concat([df_concatenado, df_score], axis=1)

In [34]:
#url="https://firebasestorage.googleapis.com/v0/b/txd-scex-tmgt-geosort-prod.appspot.com/o/delivery%2Fjavillalobos%40pipau.cl%2F149045394625.png?alt=media&token=335839a5-d0f5-4e5a-bff7-206fac166528"
#url = 'https://firebasestorage.googleapis.com/v0/b/txd-scex-tmgt-geosort-prod.appspot.com/o/delivery%2Fvirivera%40pipau.cl%2F149045744368.png?alt=media&token=6eb175fd-03a4-4338-aaa8-35d3d26cf7eb'
url = 'https://prdadessacorptrl.blob.core.windows.net/cl-images/20220602115520-Falabella-149056660575-1-EVIDENCE.png'
jason_path_api= "clave2\\tc-sc-bi-bigdata-corp-tsod-dev-739359c1cf2a.json"
jason_path_automl="key\\tc-sc-bi-bigdata-corp-tsod-dev-82c15beedd2a.json"
full_model(url,jason_path_api, jason_path_automl)

error1:

error2:

[mid: "/m/01g317"
name: "Person"
score: 0.8958184123039246
bounding_poly {
  normalized_vertices {
    x: 0.007841200567781925
    y: 0.10710307210683823
  }
  normalized_vertices {
    x: 0.7015059590339661
    y: 0.10710307210683823
  }
  normalized_vertices {
    x: 0.7015059590339661
    y: 0.9971752762794495
  }
  normalized_vertices {
    x: 0.007841200567781925
    y: 0.9971752762794495
  }
}
, mid: "/m/02dl1y"
name: "Hat"
score: 0.889347493648529
bounding_poly {
  normalized_vertices {
    x: 0.12361770868301392
    y: 0.11400888115167618
  }
  normalized_vertices {
    x: 0.42424580454826355
    y: 0.11400888115167618
  }
  normalized_vertices {
    x: 0.42424580454826355
    y: 0.26551955938339233
  }
  normalized_vertices {
    x: 0.12361770868301392
    y: 0.26551955938339233
  }
}
, mid: "/m/07mhn"
name: "Pants"
score: 0.782334566116333
bounding_poly {
  normalized_vertices {
    x: 0.0697099044919014
    y: 0.777414858341217
  }
  normalized_vertices {
 

,Prob. Producto,Producto,Prob. Sin rostro cliente,Sin rostro cliente,Contexto,Prob. Etiqueta producto,Etiqueta producto,Prob. Numero domicilio,Numero domicilio,Score
0,0.600483,1.0,0.281019,0.0,0.242553,1.0,1.0,0.0,0.0,0.60512
